In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras import Model
import os

In [7]:
drive_dir = 'drive/MyDrive'

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
model_dir = os.path.join(drive_dir,'DriveFire/model/facenet_keras.h5') 
weights_dir = os.path.join(drive_dir,'DriveFire/weights/facenet_keras_weights.h5')
pre_trained_model = load_model(model_dir)
pre_trained_model.load_weights(weights_dir)

for layer in pre_trained_model.layers:
  layer.trainable = False

pre_trained_model.summary()

Model: "inception_resnet_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
________________________________________________________________________________

In [21]:
last_layer = pre_trained_model.get_layer('Bottleneck_BatchNorm')
last_output = last_layer.output

output = tf.keras.layers.Dense(52, activation='softmax')(last_output)

model = Model( pre_trained_model.input, output )
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
____________________________________________________________________________________________

In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = os.path.join(drive_dir, 'phlwn/train')
val_dir = os.path.join(drive_dir, 'phlwn/val')

train_datagen = ImageDataGenerator(rescale = 1./255.)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    class_mode='categorical'
) 

val_datagen = ImageDataGenerator(rescale = 1./255.)

validation_generator = train_datagen.flow_from_directory(
    val_dir,
    class_mode='categorical'
)

model.fit(
    train_generator,
    # steps_per_epoch=train_generator.samples // 10,
    validation_data=validation_generator,
    # validation_steps=validation_generator.samples // 10,
    epochs=100
)

Found 468 images belonging to 52 classes.
Found 80 images belonging to 52 classes.
Epoch 1/100
15/15 [==============================] - 147s 9s/step - loss: 4.1556 - acc: 0.0192 - val_loss: 4.1432 - val_acc: 0.0125
Epoch 2/100
15/15 [==============================] - 82s 6s/step - loss: 3.9989 - acc: 0.0256 - val_loss: 4.0216 - val_acc: 0.0125
Epoch 3/100
15/15 [==============================] - 82s 6s/step - loss: 3.8915 - acc: 0.0278 - val_loss: 3.9314 - val_acc: 0.0125
Epoch 4/100
15/15 [==============================] - 82s 6s/step - loss: 3.8084 - acc: 0.0534 - val_loss: 3.8631 - val_acc: 0.0125
Epoch 5/100
15/15 [==============================] - 82s 6s/step - loss: 3.7380 - acc: 0.0662 - val_loss: 3.8107 - val_acc: 0.0375
Epoch 6/100
15/15 [==============================] - 82s 6s/step - loss: 3.6735 - acc: 0.0919 - val_loss: 3.7586 - val_acc: 0.0625
Epoch 7/100
15/15 [==============================] - 82s 6s/step - loss: 3.6104 - acc: 0.1111 - val_loss: 3.7158 - val_acc: 0.1000